In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                type_ = html.xpath('//div[@class="part__canada-ligne"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                part_number = html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_div = html.xpath('//div[@class="part__nom-spec-valeur-spec"]')
                
                # = = = = = = = = = = = = = = =
                
                src = 'https://ecat.spectrapremium.com/' + html.xpath('//div[@class="part__zoom-gallery"]/descendant::img/@src')[0].strip().replace('/imagettes/', '/').replace('_220.', '.')

                # = = = = = = = = = = = = = = =
                
                vehicle_page = 1 if len(html.xpath('//li[@class="paginator__last"]')) == 0 else int(html.xpath('//li[@class="paginator__last"]/a/@href')[0].split('#')[0].split('=')[-1])

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Type': type_,
                                         'Part_Number': part_number,
                                         'Vehicle': '',
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src,
                                         'Vehicle_Page': vehicle_page}])

                for div in list_div:
                    df_temp.loc[0, div.xpath('./dt/text()')[0].strip()] = div.xpath('./dd/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：553

[ok] - https://ecat.spectrapremium.com/en/parts/BMP03A
[尝试次数：1] - [剩余数量：543] - [当前时间：10:51:39]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP13A
[尝试次数：1] - [剩余数量：542] - [当前时间：10:51:39]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP09A
[尝试次数：1] - [剩余数量：541] - [当前时间：10:51:39]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：2] - [剩余数量：540] - [当前时间：10:51:40]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP08A
[尝试次数：1] - [剩余数量：539] - [当前时间：10:51:40]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP11A
[尝试次数：1] - [剩余数量：538] - [当前时间：10:51:40]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP15A
[尝试次数：1] - [剩余数量：537] - [当前时间：10:51:41]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP20A
[尝试次数：1] - [剩余数量：536] - [当前时间：10:51:41]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP02A
[尝试次数：5] - [剩余数量：535] - [当前时间：10:51:43]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP23A
[尝试次数：2] - [剩余数量：534] - [当前时间：10:51:43]

[ok] - https://ecat.spectraprem

[ok] - https://ecat.spectrapremium.com/en/parts/CRP69A
[尝试次数：1] - [剩余数量：457] - [当前时间：10:52:12]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP66A
[尝试次数：2] - [剩余数量：456] - [当前时间：10:52:12]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP64A
[尝试次数：2] - [剩余数量：455] - [当前时间：10:52:13]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP72A
[尝试次数：1] - [剩余数量：454] - [当前时间：10:52:13]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP73A
[尝试次数：1] - [剩余数量：453] - [当前时间：10:52:13]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP81A
[尝试次数：1] - [剩余数量：452] - [当前时间：10:52:14]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP76A
[尝试次数：1] - [剩余数量：451] - [当前时间：10:52:14]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP75A
[尝试次数：4] - [剩余数量：450] - [当前时间：10:52:15]

[ok] - https://ecat.spectrapremium.com/en/parts/DTP02A
[尝试次数：1] - [剩余数量：449] - [当前时间：10:52:15]

[ok] - https://ecat.spectrapremium.com/en/parts/DTP01A
[尝试次数：2] - [剩余数量：448] - [当前时间：10:52:16]

[ok] - https://ecat.spectrapremium.com/e

[ok] - https://ecat.spectrapremium.com/en/parts/FP75A
[尝试次数：3] - [剩余数量：369] - [当前时间：10:52:53]

[ok] - https://ecat.spectrapremium.com/en/parts/FP70A
[尝试次数：8] - [剩余数量：368] - [当前时间：10:52:54]

[ok] - https://ecat.spectrapremium.com/en/parts/FP72A
[尝试次数：5] - [剩余数量：367] - [当前时间：10:52:54]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP10A
[尝试次数：2] - [剩余数量：366] - [当前时间：10:52:54]

[ok] - https://ecat.spectrapremium.com/en/parts/FP78A
[尝试次数：1] - [剩余数量：365] - [当前时间：10:52:54]

[ok] - https://ecat.spectrapremium.com/en/parts/FP64A
[尝试次数：3] - [剩余数量：364] - [当前时间：10:52:56]

[ok] - https://ecat.spectrapremium.com/en/parts/FP76A
[尝试次数：5] - [剩余数量：363] - [当前时间：10:52:56]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP14A
[尝试次数：7] - [剩余数量：362] - [当前时间：10:52:56]

[ok] - https://ecat.spectrapremium.com/en/parts/FP80A
[尝试次数：2] - [剩余数量：361] - [当前时间：10:52:56]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP05A
[尝试次数：7] - [剩余数量：360] - [当前时间：10:52:57]

[ok] - https://ecat.spectrapremium.com/en/parts

[ok] - https://ecat.spectrapremium.com/en/parts/GMP63B
[尝试次数：2] - [剩余数量：282] - [当前时间：10:53:25]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP59A
[尝试次数：6] - [剩余数量：281] - [当前时间：10:53:25]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP64A
[尝试次数：5] - [剩余数量：280] - [当前时间：10:53:25]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP66A
[尝试次数：4] - [剩余数量：279] - [当前时间：10:53:26]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP66B
[尝试次数：1] - [剩余数量：278] - [当前时间：10:53:26]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP71A
[尝试次数：1] - [剩余数量：277] - [当前时间：10:53:26]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP56A
[尝试次数：15] - [剩余数量：276] - [当前时间：10:53:26]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP68A
[尝试次数：4] - [剩余数量：275] - [当前时间：10:53:27]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP69A
[尝试次数：1] - [剩余数量：274] - [当前时间：10:53:27]

[ok] - https://ecat.spectrapremium.com/en/parts/GMP70A
[尝试次数：4] - [剩余数量：273] - [当前时间：10:53:27]

[ok] - https://ecat.spectrapremium.com/

[ok] - https://ecat.spectrapremium.com/en/parts/HYP17A
[尝试次数：1] - [剩余数量：196] - [当前时间：10:53:55]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP12A
[尝试次数：3] - [剩余数量：195] - [当前时间：10:53:56]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP14A
[尝试次数：6] - [剩余数量：194] - [当前时间：10:53:56]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP01A
[尝试次数：23] - [剩余数量：193] - [当前时间：10:53:57]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP09A
[尝试次数：11] - [剩余数量：192] - [当前时间：10:53:57]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP06A
[尝试次数：16] - [剩余数量：191] - [当前时间：10:53:58]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP19A
[尝试次数：4] - [剩余数量：190] - [当前时间：10:53:58]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP20A
[尝试次数：2] - [剩余数量：189] - [当前时间：10:53:58]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP18A
[尝试次数：9] - [剩余数量：188] - [当前时间：10:53:59]

[ok] - https://ecat.spectrapremium.com/en/parts/HYP27A
[尝试次数：2] - [剩余数量：187] - [当前时间：10:53:59]

[ok] - https://ecat.spectrapremium.co

[ok] - https://ecat.spectrapremium.com/en/parts/NSP15A
[尝试次数：19] - [剩余数量：110] - [当前时间：10:54:25]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP28A
[尝试次数：8] - [剩余数量：109] - [当前时间：10:54:25]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP27A
[尝试次数：11] - [剩余数量：108] - [当前时间：10:54:25]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP34A
[尝试次数：4] - [剩余数量：107] - [当前时间：10:54:26]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP38A
[尝试次数：3] - [剩余数量：106] - [当前时间：10:54:27]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP29A
[尝试次数：11] - [剩余数量：105] - [当前时间：10:54:27]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP26C
[尝试次数：15] - [剩余数量：104] - [当前时间：10:54:27]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP40A
[尝试次数：3] - [剩余数量：103] - [当前时间：10:54:27]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP35A
[尝试次数：8] - [剩余数量：102] - [当前时间：10:54:28]

[ok] - https://ecat.spectrapremium.com/en/parts/NSP39A
[尝试次数：5] - [剩余数量：101] - [当前时间：10:54:28]

[ok] - https://ecat.spectrapremium.c

[ok] - https://ecat.spectrapremium.com/en/parts/TOP73A
[尝试次数：25] - [剩余数量：22] - [当前时间：10:55:00]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP26A
[尝试次数：4] - [剩余数量：21] - [当前时间：10:55:01]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP25A
[尝试次数：5] - [剩余数量：20] - [当前时间：10:55:01]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP22A
[尝试次数：2] - [剩余数量：19] - [当前时间：10:55:02]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP28A
[尝试次数：4] - [剩余数量：18] - [当前时间：10:55:02]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP27A
[尝试次数：5] - [剩余数量：17] - [当前时间：10:55:03]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP32A
[尝试次数：1] - [剩余数量：16] - [当前时间：10:55:03]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP23A
[尝试次数：3] - [剩余数量：15] - [当前时间：10:55:03]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP20A
[尝试次数：17] - [剩余数量：14] - [当前时间：10:55:03]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP37A
[尝试次数：2] - [剩余数量：13] - [当前时间：10:55:04]

[ok] - https://ecat.spectrapremium.com/en/parts/